# Here I've used two methods to convert the raw json files to tabular format
1. Exploring the json file and filtering the required data from the key,values,list,str and lists by selecting specific identity, the next step is creating a function and passing the filtered data so that when we pass the function we get the required data in the tabular format. The next step is to loop every json file from the folder to append in the dataframe.
2. Recursion method: The function here checks every keys (whether its in int,dict,list,str) it gets its corrseponding values in the entire file. I would suggest the 1st method, because Recursion method is too time consuming.

In [2]:
#Importing necessary libraries and packages
import numpy as np  
import pandas as pd
import os
import json

In [3]:
#Changing the working directory
os.getcwd()

'C:\\Users\\Dell'

In [4]:
os.chdir('C:\\Users\Dell\\Desktop\\New folder\\exa-data-eng-assessment-main')

In [5]:
# importing single json file and exploring the keys and values 
input_file = json.load(open('data\Aaron697_Dickens475_8c95253e-8ee8-9ae8-6d40-021d702dc78e.json'))

In [9]:
# keys of the json file
input_file.keys()

dict_keys(['resourceType', 'type', 'entry'])

In [11]:
# exploring the values inside the key(entry)
fitr_df = input_file['entry']


In [12]:
#Exploring resource type Patient to extract the important data
rsrc = fitr_df[0]['resource']
rsrc

{'resourceType': 'Patient',
 'id': '8c95253e-8ee8-9ae8-6d40-021d702dc78e',
 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: master-branch-latest\n .   Person seed: 8886041252665297312  Population seed: 13235</div>'},
 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2106-3',
      'display': 'White'}},
    {'url': 'text', 'valueString': 'White'}]},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2186-5',
      'display': 'Not Hispanic or Latino'}},
    {'url': 

In [8]:
#filtering out the family and given name 
fitr_df[0]['resource']['name']

[{'use': 'official',
  'family': 'Dickens475',
  'given': ['Aaron697'],
  'prefix': ['Mr.']}]

In [9]:
#extracting the patient_id
patient_id = fitr_df[0]['resource']['id']
patient_id

'8c95253e-8ee8-9ae8-6d40-021d702dc78e'

In [13]:
#Race of the particular patient
race_col_name = rsrc['extension'][0]['url'].split('-')[-1]
race_val = rsrc['extension'][0]['extension'][-1]['valueString']
print(race_col_name)
print(race_val)

race
White


In [18]:
#Taking out the measured value of the patient
text =fitr_df[2]['resource']['code']
text = pd.DataFrame(text,index=[0])
text.drop(['coding'],inplace=True,axis=1)
text

,text
0,Body Height


In [19]:
#Value and unit of the value
value_df=fitr_df[2]['resource']['valueQuantity']
value_df = pd.DataFrame(value_df,index=[0])
value_df.drop(['system','code'],inplace=True,axis=1)
value_df

,value,unit
0,53.8,cm


In [13]:
#Patient location details
addrs_dic = rsrc['extension'][4]['valueAddress']
addrs_dic

{'city': 'Barnstable', 'state': 'Massachusetts', 'country': 'US'}

In [14]:
#Extracting Patient_id 
patient_id = fitr_df[0]['resource']['id']
patient_id

'8c95253e-8ee8-9ae8-6d40-021d702dc78e'

In [15]:
#Extracting patient DOB
dob = fitr_df[0]['resource']['birthDate']
dob

'1944-08-28'

In [18]:
#After exploring the json data now we are creating a new function by passing the json as input and
#it returns back with the filtered and extracted data from the file
def update(raw_input):
    fitr_df = raw_input['entry']
    rsrc = fitr_df[0]['resource']
    addrs_dic = rsrc['extension'][4]['valueAddress']
    print(addrs_dic)
    race_col_name = rsrc['extension'][0]['url'].split('-')[-1]
    race_val = rsrc['extension'][0]['extension'][-1]['valueString']
    patient_id = fitr_df[0]['resource']['id']
    name_dic=fitr_df[0]['resource']['name'][0]
    dob = fitr_df[0]['resource']['birthDate']
    print(name_dic)
    df_dic = dict()
    df_dic.update(addrs_dic)
    df_dic.update({race_col_name:race_val})
    df_dic.update({'Patient_id':patient_id})
    df_dic.update({'DOB':dob})
    df_dic.update(name_dic)
    df = pd.DataFrame([df_dic])
    # dropping the columns which are not required or important
    df.drop(['use'],inplace=True,axis=1)
    #Renaming columns 
    df.rename(columns={'family':'Family_Name', 'given':'Patient_Name'},inplace=True)
    print(df_dic)
    #Adding a logic, if prefix values is NAN fill it with '-' else the data remains the same
    #here we could see len(name_dic) is 4 when prefix is available for the respective patient, so if the
    #length is lesser than 4 the prefix value is filled with '-' since there is no NAN values in remaining columns
    if len(name_dic) == 4:
        #Removing the [] brackets in the data
        df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
        df['prefix'] = df['prefix'].str.replace("'","")
    elif len(name_dic)<4:
        df['prefix']='-'
    df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
    #removing the quotes from the given name
    df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
    df['Patient_Name'] = df['Patient_Name'].str.replace("'","")
    df['Family_Name'] = df['Family_Name'].str.replace('\d',"")
    value_df=fitr_df[2]['resource']['valueQuantity']
    value_df = pd.DataFrame(value_df,index=[0])
    #removing the unwanted columns
    value_df.drop(['system','code'],inplace=True,axis=1)
    text =fitr_df[2]['resource']['code']
    text = pd.DataFrame(text,index=[0])
    text.drop(['coding'],inplace=True,axis=1)
    #joining the dataframe to make it easier
    df = pd.concat([df, value_df],axis=1)
    df = pd.concat([df, text],axis=1)
    #Rearraning the column names as per the specification
    df=df[['Patient_id','prefix','Patient_Name','Family_Name','DOB','text',
               'value','unit','city','state','country','race']]
    #df=df[['Patient_id','prefix','Patient_Name','Family_Name','DOB',
                #'city','state','country','race']]
    return df

In [19]:
#Creating a new empty dataframe
mdf = pd.DataFrame()
#Looping and loading every json file in the folder
for f in os.listdir('data'):
    #print(f)
    raw = json.load(open('data/'+f,encoding="utf8"))
    #Passing the input file to the above defined function
    rdf = update(raw)
    #print(rdf)
    #Concatenating or appending the empty dataframe with the output dataframe of the function
    mdf = pd.concat([rdf,mdf],axis=0)
    mdf = mdf.reset_index(drop=True)

{'city': 'Barnstable', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Dickens475', 'given': ['Aaron697'], 'prefix': ['Mr.']}
{'city': 'Barnstable', 'state': 'Massachusetts', 'country': 'US', 'race': 'White', 'Patient_id': '8c95253e-8ee8-9ae8-6d40-021d702dc78e', 'DOB': '1944-08-28', 'use': 'official', 'family': 'Dickens475', 'given': ['Aaron697'], 'prefix': ['Mr.']}


C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
C:\Users\Dell\AppData\Loca

{'city': 'Douglas', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Jerde200', 'given': ['Aaron697'], 'prefix': ['Mr.']}
{'city': 'Douglas', 'state': 'Massachusetts', 'country': 'US', 'race': 'White', 'Patient_id': '6fa23508-960e-ff22-c3d0-0519a036543b', 'DOB': '2001-06-28', 'use': 'official', 'family': 'Jerde200', 'given': ['Aaron697'], 'prefix': ['Mr.']}


C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
C:\Users\Dell\AppData\Loca

{'city': 'Bourne', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Price929', 'given': ['Abbey813'], 'prefix': ['Ms.']}
{'city': 'Bourne', 'state': 'Massachusetts', 'country': 'US', 'race': 'Black or African American', 'Patient_id': '83524678-9bff-93b7-ef89-d7f5390072ff', 'DOB': '1980-06-10', 'use': 'official', 'family': 'Price929', 'given': ['Abbey813'], 'prefix': ['Ms.']}


...

C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
C:\Users\Dell\AppData\Loca

{'city': 'Concord', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Barton704', 'given': ['Santiago500'], 'prefix': ['Mr.']}
{'city': 'Concord', 'state': 'Massachusetts', 'country': 'US', 'race': 'White', 'Patient_id': '686e4b2e-1f8d-7018-c335-1604a300121b', 'DOB': '1976-10-22', 'use': 'official', 'family': 'Barton704', 'given': ['Santiago500'], 'prefix': ['Mr.']}


C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
C:\Users\Dell\AppData\Loca

{'city': 'Haverhill', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Wyman904', 'given': ['Zachery872'], 'prefix': ['Mr.']}
{'city': 'Haverhill', 'state': 'Massachusetts', 'country': 'US', 'race': 'White', 'Patient_id': '37d57af4-d27f-fb97-9b5c-a1090f5a57b7', 'DOB': '1970-01-04', 'use': 'official', 'family': 'Wyman904', 'given': ['Zachery872'], 'prefix': ['Mr.']}
{'city': 'Haverhill', 'state': 'Massachusetts', 'country': 'US'}
{'use': 'official', 'family': 'Schoen8', 'given': ['Zane918'], 'prefix': ['Mr.']}
{'city': 'Haverhill', 'state': 'Massachusetts', 'country': 'US', 'race': 'White', 'Patient_id': '4995ca99-2ab9-3b18-a56e-c33d29e53af0', 'DOB': '1950-08-27', 'use': 'official', 'family': 'Schoen8', 'given': ['Zane918'], 'prefix': ['Mr.']}


C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['prefix'] = df['prefix'].astype(str).astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Patient_Name'] = df['Patient_Name'].astype(str).str.replace('[','').astype(str).str.replace(']','')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14500\2770688869.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Patient_Name'] = df['Patient_Name'].str.replace('\d',"")
C:\Users\Dell\AppData\Loca

In [32]:
#Tried to pull out the patient's physical measurement value(height,weight), but each file the index of 'ValueQuantity' 
#are in different index 

mdf

,Patient_id,prefix,Patient_Name,Family_Name,DOB,text,value,unit,city,state,country,race
0,64cdd7b0-d5a5-ca8a-4b03-3db12d7534be,-,Cherie,Reinger,2014-09-29,Body Height,53.8,cm,Taunton,Massachusetts,US,White


# The final output file with the details of the patient in the tabular format

In [35]:
mdf

,Patient_id,prefix,Patient_Name,Family_Name,DOB,city,state,country,race
0,4995ca99-2ab9-3b18-a56e-c33d29e53af0,Mr.,Zane,Schoen,1950-08-27,Haverhill,Massachusetts,US,White
1,37d57af4-d27f-fb97-9b5c-a1090f5a57b7,Mr.,Zachery,Wyman,1970-01-04,Haverhill,Massachusetts,US,White
2,686e4b2e-1f8d-7018-c335-1604a300121b,Mr.,Santiago,Barton,1976-10-22,Concord,Massachusetts,US,White
3,f38fe4a9-1634-144c-0147-53c0300c1561,Mr.,Sandy,Conn,1969-04-01,Boston,Massachusetts,US,White
4,15201e77-7e36-0667-73bc-013783e57649,Mrs.,Sandee,Hodkiewicz,1946-02-17,Beijing,Beijing Municipality,CN,Asian
...,...,...,...,...,...,...,...,...,...
74,6e4ac285-2a8d-a30d-5ecb-e32cb595a876,Mrs.,Bette,Anderson,1968-07-07,Hanover,Massachusetts,US,White
75,4e343b0a-8698-b6dd-64c6-c2d2d0959e6e,Mrs.,Beth,Hansen,1946-05-21,Medford,Massachusetts,US,White
76,83524678-9bff-93b7-ef89-d7f5390072ff,Ms.,Abbey,Price,1980-06-10,Bourne,Massachusetts,US,Black or African American
77,6fa23508-960e-ff22-c3d0-0519a036543b,Mr.,Aaron,Jerde,2001-06-28,Douglas,Massachusetts,US,White


In [36]:
#Exporting the file into csv to the local machine
mdf.to_csv('output_file.csv')

# Recursion method - a defined function can call itself.

## Testing with just one json file, because this function tracks every keys (whether it's inside the array or any other format like int,str, list etc.) from the file and adds to df as an column

In [6]:
# loading json file
dc = json.load(open('data\Aaron697_Dickens475_8c95253e-8ee8-9ae8-6d40-021d702dc78e.json'))
#defining a new function 
def create_dic(d,ck):
    if type(d)==dict:
        for k in d.keys():
            #print(k)
            if type(d[k])==str:
                #print('str')
                #here we have multiple keys with same name to differentiate that we are splitting the name and adding the no of occurences
                dcl_ls = [s for s in df_dict.keys() if s.split('_')[0]==k]
                dcl = str(len(dcl_ls))
                #print(dcl_ls)
                df_dict[k+'_'+dcl] = d[k]
            #this elif method is used to handle if the values inside are list 
            elif type(d[k])==list:
                #print('list')
                for c in d[k]:
                    create_dic(c,k)
            #this elif method is used to handle if the values inside are dict 
            elif type(d[k])==dict:
                #print('dict')
                create_dic(d[k],k)
    else:
        dcl_ls = [s for s in df_dict.keys() if s.split('_')[0]==ck]
        dcl = str(len(dcl_ls))
        df_dict[ck+'_'+dcl] = d

In [7]:
#create a empty dict to append the results
df_dict = dict()

In [8]:
create_dic(dc,'1')

In [44]:
df_dict.keys()

dict_keys(['resourceType_0', 'type_0', 'fullUrl_0', 'resourceType_1', 'id_0', 'profile_0', 'status_0', 'div_0', 'url_0', 'url_1', 'system_0', 'code_0', 'display_0', 'url_2', 'valueString_0', 'url_3', 'url_4', 'system_1', 'code_1', 'display_1', 'url_5', 'valueString_1', 'url_6', 'valueString_2', 'url_7', 'valueCode_0', 'url_8', 'city_0', 'state_0', 'country_0', 'url_9', 'url_10', 'system_2', 'value_0', 'system_3', 'code_2', 'display_2', 'text_0', 'system_4', 'value_1', 'system_5', 'code_3', 'display_3', 'text_1', 'system_6', 'value_2', 'system_7', 'code_4', 'display_4', 'text_2', 'system_8', 'value_3', 'system_9', 'code_5', 'display_5', 'text_3', 'system_10', 'value_4', 'use_0', 'family_0', 'given_0', 'prefix_0', 'system_11', 'value_5', 'use_1', 'gender_0', 'birthDate_0', 'deceasedDateTime_0', 'url_11', 'url_12', 'url_13', 'line_0', 'city_1', 'state_1', 'country_1', 'system_12', 'code_6', 'display_6', 'text_4', 'system_13', 'code_7', 'display_7', 'text_5', 'method_0', 'url_14', 'fullUrl

### This step is to fetch the unique columns

In [40]:
#since we have many duplicate keys in the dictionary, we need to remove them
kls = []
for k in df_dict.keys():
    k_str = k.split('_')[0]
    if k_str not in kls:
        kls.append(k_str)

In [41]:
kls

['resourceType',
 'type',
 'fullUrl',
 'id',
 'profile',
 'status',
 'div',
 'url',
 'system',
 'code',
 'display',
 'valueString',
 'valueCode',
 'city',
 'state',
 'country',
 'value',
 'text',
 'use',
 'family',
 'given',
 'prefix',
 'gender',
 'birthDate',
 'deceasedDateTime',
 'line',
 'method',
 'reference',
 'start',
 'end',
 'onsetDateTime',
 'recordedDate',
 'effectiveDateTime',
 'issued',
 'contentType',
 'data',
 'date',
 'created',
 'diagnosisSequence',
 'currency',
 'intent',
 'outcome',
 'abatementDateTime',
 'unit',
 'occurrenceDateTime',
 'informationSequence',
 'procedureSequence',
 'authoredOn',
 'started',
 'uid',
 'title',
 'recorded']

### Creating a new function passing df as input, this is for postprocessing the df
1. Renaming the columns
2. Dropping the columns which are not required to this case(by adding a list of column names called prefixes)

In [42]:
def postprocess(df):
    df.rename(columns={'city_0': 'City','state_0':'State','country_0':'Country','use_0':'Use','family_0':'Family_name',
                     'given_0':'Given_name','prefix_0':'Prefix','use_1':'Use','gender_0':'Gender','birthDate_0':'DOB',
                     'deceasedDateTime_0':'DeceasedDateTime','id_0':'Patient_id'}
                    ,inplace=True)
    prefixes = ['city','state','uid','recorded','url','state','countr','use','issue','curren','date','abatementDateTime','unit','occurrenceDateTime', 'informationSequence','procedureSequence','onsetDateTime','recordedDate',
 'effectiveDateTime','type','fullUrl','profile',
 'status','code',
 'display',
 'valueString',
 'valueCode','line',
 'method',
 'reference',
 'start',
 'end','issued',
 'contentType',
 'data',
 'date',
 'created',
 'diagnosisSequence','intent',
 'outcome',
 'abatementDateTime',
 'unit',
 'occurrenceDateTime',
 'informationSequence',
 'procedureSequence','authoredOn',
 'started',
 'uid','system','id','res','val','tex','div','title']
    unwanted = df.columns[df.columns.str.startswith (tuple(prefixes))]
    df.drop(unwanted, axis=1, inplace=True)
    df.shape
    df.to_excel('new.xlsx')
    return df

In [45]:
adf = pd.DataFrame([df_dict])

#### Here we can see that we have 20k columns from the json file after passing the recursion function

In [46]:
adf.shape

(1, 20562)

#### Now after passing through the post_processing we can able to see just 12 columns, we can even add more columns by editing the prefixes list from the above function

In [47]:
postprocess(adf)

,Patient_id,City,State,Country,Use,Family_name,Given_name,Prefix,Use,Gender,DOB,DeceasedDateTime
0,8c95253e-8ee8-9ae8-6d40-021d702dc78e,Barnstable,Massachusetts,US,official,Dickens475,Aaron697,Mr.,home,male,1944-08-28,1998-08-15T13:05:53+01:00


In [49]:
adf.shape

(1, 12)